<a href="https://colab.research.google.com/github/rosanth/Recommended_Sys/blob/Sajeevan/Reco_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf

In [ ]:
import fitz  #
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Dropout
import numpy as np
import re

In [19]:
# Open the PDF file
doc = fitz.open("Sajeevan.pdf")
out = open("output.txt", "w", encoding="utf-8")  # Open output file in text mode

# Extract text from PDF
for page in doc:
    text = page.get_text("text")  # Extract text
    out.write(text + "\n")  # Write text with newline for separation
out.close()

# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w.,]', ' ', text)
    return text.strip()

# Read the file
with open("output.txt", "r", encoding="utf-8") as file:
    resume_text = file.read()

# Clean the text
resume_text = clean_text(resume_text)
print(resume_text)

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Sample job requirement
job_requirements = """
Looking for a candidate with knowledge in Python, Java, and Power BI.
Required skills:
- Python Programming
- Java Development
- Data Visualization (Power BI)
- Database
"""
job_requirements = preprocess_text(job_requirements)

# Tokenization
max_tokens = 1000
max_length = 100
vectorizer = TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_length)
vectorizer.adapt([job_requirements])

# Model
model = keras.Sequential([
    Embedding(input_dim=max_tokens, output_dim=32, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert job_requirements t
X_train = vectorizer([job_requirements])
y_train = np.array([1])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1)

# Function to evaluate a resume
def evaluate_resume(resume_text):
    resume_text = preprocess_text(resume_text)
    input_data = vectorizer([resume_text])
    prediction = model.predict(input_data)[0][0]
    return "Resume is suitable for the job." if prediction > 0.6 else "Resume is not suitable for the job."

# Example usage
with open("output.txt", "r", encoding="utf-8") as file:
    preprocessed_text = preprocess_text(file.read())
    print(evaluate_resume(preprocessed_text))


Sajeevan Jeyakumar sajeevanoffi gmail.com  94753714767 131, Hospital Front Lane, Chenkalady linkedin.com in sajeevan  jeyakumar 987ba91ba PROFILE Enthusiastic and hardworking graduate seeking an entry level position to utilize my skills, gain experience, and contribute to the growth of the organization while advancing my personal and professional development LANGUAGES Tamil English EDUCATION Higher Diploma in Software Engineering Wayamba University of Srilanka present Advanced Level BT KK Chenkalady Central College 08 2016   Chenkalady, Srilanka Ordinary Level BT KK Chenkalady Central College 12 2013   Chenakalady, Srilanka SKILLS Python  Python Programming, Data Structures, Web Scraping, API Interaction, Data Manipulation, File Handling Java  Java Programming, Object Oriented Design, GUI Development, File Handling, Exception Handling, Database Connectivity Power BI  Data Visualization, Data Modeling, Using DAX, Report Customization, Interactive Capabilities MS Office  Microsoft Excel,